<a href="https://colab.research.google.com/github/wahyunh10/Car-Auction-Prices-Project/blob/main/Car_Auction_Prices_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Car Auction Prices Project**

1. Role: Sebagai tim data scientist dalam suatu perusahaan bernama PT.Dats Your Future yang berperan sebagai dealer jual beli mobil bekas yang bertanggung jawab untuk memberikan rekomendasi harga mobil bekas berdasarkan data histori pembelian 2 tahun sebelumnya yang ada di Amerika yang diasumsikan ideal karena mobil berhasil dibeli dan dijual kembali ke konsumen dengan harga ideal.
2. Masalah:
  * Perusahaan menentukan harga beli mobil bekas tidak terstandar (tidak sesuai kondisi mobil sebenarnya) sehingga mendapat harga mobil bekas yang berbeda dari perspektif yang bias oleh setiap staff.
   * Proses screening dan negosiasi yang cukup lama akhirnya customer menjadi ragu dan cenderung tidak menjual mobil nya ke perusahaan kita.
3. Goal: Bisa membantu perusahaan menentukan harga beli mobil bekas yang dapat terstandarkan berdasarkan model machine learning.
4. Objective: Membuat tools / model machine learning untuk membantu memprediksi harga mobil berdasarkan kondisi sebenarnya.
5. usiness Metrics: Sales Pembelian dan Efisiensi Waktu

# **Final Project - EDA, Insight, Visualization**

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# **Mengubah parameter default matplotlib**
Parameter default matplotlib dapat diubah dengan rcParams sebagai berikut

In [6]:
from matplotlib import rcParams

rcParams['figure.figsize'] = 12, 4
rcParams['lines.linewidth'] = 3
rcParams['xtick.labelsize'] = 'x-large'
rcParams['ytick.labelsize'] = 'x-large'

# **Load dataset**
Kita akan melihat dataset prediksi harga mobil

The dataset contains historical car auction sales prices, scraped from the outside internet sources collected in 2015. This dataset taken from kaggle (https://www.kaggle.com/tunguz/used-car-auction-prices).

This dataset contains 491641 rows and 15 columns:

* year : information of year production of the car
* make : brand of the car
* model : model of the brand car
* trim : spesific model of the car
* body : body type of the car
* transmission : transmission type of the car
* vin : unique id of the car
* state : code state where the transaction happened
* condition : rating condition of the car
* odometer : total distance of the car
* color : body color of the car
* interior : interior color of the car
* seller : place that sold the car
* mmr : market price of the car
* sellingprice : price of the car
* saledate : date when the car sold

In [7]:
df = pd.read_csv("car_prices.csv")
df.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate,Unnamed: 16
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,"kia motors america, inc",20500,21500,Tue Dec 16 2014 12:30:00 GMT-0800 (PST),NaN
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,"kia motors america, inc",20800,21500,Tue Dec 16 2014 12:30:00 GMT-0800 (PST),NaN
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,4.5,1331.0,gray,black,financial services remarketing (lease),31900,30000,Thu Jan 15 2015 04:30:00 GMT-0800 (PST),NaN
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,4.1,14282.0,white,black,volvo na rep/world omni,27500,27750,Thu Jan 29 2015 04:30:00 GMT-0800 (PST),NaN
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,4.3,2641.0,gray,black,financial services remarketing (lease),66000,67000,Thu Dec 18 2014 12:30:00 GMT-0800 (PST),NaN


# **Descriptive Statistics**

Hal pertama yang biasanya dilakukan dalam EDA adalah mengecek tipe data setiap kolom dan jumlah nilai yang hilang. Hal ini bisa kita lakukan dengan menggunakan df.info().

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558837 entries, 0 to 558836
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          558837 non-null  int64  
 1   make          548536 non-null  object 
 2   model         548438 non-null  object 
 3   trim          548186 non-null  object 
 4   body          545642 non-null  object 
 5   transmission  493484 non-null  object 
 6   vin           558833 non-null  object 
 7   state         558837 non-null  object 
 8   condition     547043 non-null  object 
 9   odometer      558743 non-null  float64
 10  color         558088 non-null  object 
 11  interior      558088 non-null  object 
 12  seller        558837 non-null  object 
 13  mmr           558837 non-null  object 
 14  sellingprice  558837 non-null  int64  
 15  saledate      558837 non-null  object 
 16  Unnamed: 16   25 non-null      object 
dtypes: float64(1), int64(2), object(14)
memory usage

Kita dapat melihat bahwa:

1. Dataframe memiliki total 558811 baris dan 16 kolom
2. Target output sepertinya adalah kolom `sellingprice` dengan tipe data `int64`
3. Berarti sisanya adalah feature
Dari informasi di atas, kita bisa pisahkan kolom kategorical dan numerical sebagai berikut:

Dari informasi di atas, kita bisa pisahkan kolom kategorical dan numerical sebagai berikut:

In [9]:
cats = ['make', 'model', 'trim', 'body', 'transmission','vin','state','color','interior','seller','saledate']
nums = ['year', 'condition', 'odometer', 'mmr', 'sellingprice']

# **Sampling untuk memahami data**

Setelah mengetahui kolom apa saja yang ada di dalam dataset, kita lakukan sampling untuk memastikan apakah isi kolomnya sesuai ekspektasi. Biasakan lakukan ini beberapa kali karena seringkali apabila ada keanehan tidak akan keluar pada sampling pertama.

Sampling dapat dilakukan dengan menggunakan `df.sample()`

In [ ]:
df.sample(10)

Beberapa pengamatan:

* Target kita, sellingprice benar bertipe int64